In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

In [ ]:
def multi_hot_sequences(sequences, dimension):
  results = np.zeros((len(sequences), dimension))
  for i, word_indices in enumerate(sequences):
    results[i, word_indices] = 1.0
  return results

In [ ]:
print(train_data[0])

In [ ]:
train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

In [ ]:
np.where(train_data[0] == 1.0)

In [ ]:
plt.plot(train_data[0])

In [ ]:
baseline_model = keras.Sequential([
                                   keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
                                   keras.layers.Dense(16, activation='relu'),
                                   keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])

In [ ]:
baseline_model.summary()

In [ ]:
baseline_history = baseline_model.fit(train_data, train_labels, batch_size=512, epochs=20, validation_data=(test_data, test_labels), verbose=2)

In [ ]:
smaller_model = keras.Sequential([
                                  keras.layers.Dense(4, activation='relu', input_shape=(NUM_WORDS,)),
                                  keras.layers.Dense(4, activation='relu'),
                                  keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
smaller_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])

In [ ]:
smaller_model.summary()

In [ ]:
smaller_history = smaller_model.fit(train_data, train_labels, batch_size=512, epochs=20, verbose=2, validation_data=(test_data, test_labels))

In [ ]:
bigger_model = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
bigger_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy','binary_crossentropy'])

In [ ]:
bigger_model.summary()

In [ ]:
bigger_history = bigger_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

In [ ]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16, 10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_' + key], '--', label=name.title() + ' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(), label=name.title() + ' Train')
  plt.xlabel('Epochs')
  plt.ylabel(key.replace('-', ' ').title())
  plt.legend()
  plt.xlim([0, max(history.epoch)])

In [ ]:
plot_history([('baseline', baseline_history), ('smaller', smaller_history), ('bigger', bigger_history)])

In [ ]:
l2_model = tf.keras.Sequential([
                                keras.layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001), input_shape=(NUM_WORDS,)),
                                keras.layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
                                keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
l2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])

In [ ]:
l2_model_history = l2_model.fit(train_data, train_labels, batch_size=512, epochs=20, verbose=2, validation_data=(test_data, test_labels))

In [ ]:
plot_history([('baseline', baseline_history),
              ('l2', l2_model_history)])

In [ ]:
dpt_model = keras.models.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

dpt_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

dpt_model_history = dpt_model.fit(train_data, train_labels,
                                  epochs=20,
                                  batch_size=512,
                                  validation_data=(test_data, test_labels),
                                  verbose=2)

In [ ]:
plot_history([('baseline', baseline_history),
              ('dropout', dpt_model_history)])